In [1]:
import pandas as pd

from sysdata.csv.csv_futures_contract_prices import ConfigCsvFuturesPrices
from sysinit.futures.adjustedprices_from_db_multiple_to_db import process_adjusted_prices_single_instrument
from sysinit.futures.multipleprices_from_db_prices_and_csv_calendars_to_db import process_multiple_prices_single_instrument
from sysinit.futures.norgate_futures_contract_prices import rename_files, check_prices_match, transfer_norgate_prices_to_db_single
from sysinit.futures.rollcalendars_from_arcticprices_to_csv import build_and_write_roll_calendar
from sysobjects.dict_of_futures_per_contract_prices import dictFuturesContractPrices
from sysobjects.futures_per_contract_prices import futuresContractPrices
from sysproduction.data.prices import diagPrices
from sysproduction.update_sampled_contracts import update_sampled_contracts

diag_prices = diagPrices()




Configuring sim logging
2024-09-13 16:04:27 DEBUG config {'type': 'config', 'stage': 'config'} Adding config defaults
Private configuration private/private_config.yaml does not exist; no problem if running in sim mode


## renaming files for a single instrument from Norgate to PST format
- this moves the files from `datapath` to `datapath`_conv 

In [2]:
datapath = "/home/alpha/data/norgate/Futures"
#datapath = "/Users/ageach/Documents/backup/pst_jani/norgate/Futures"

# rename_files(datapath, "ES", dry_run=True)
rename_files(datapath, "ES", dry_run=False)


Successfully mapped: []
Unmapped: []
Not properly configured in pysystemtrade: []
No roll config in pysystemtrade: []


## renaming files for ALL instruments from Norgate to PST format

In [ ]:
datapath = "/home/alpha/data/norgate/Futures"
#datapath = "/Users/ageach/Documents/backup/pst_jani/norgate/Futures"

rename_files(datapath, dry_run=True)
#rename_files(datapath, dry_run=False)

## Check that IB and Norgate prices match
- sometimes one is in dollars, the other cents etc
- If there is a mismatch, make a record of the instrument code and skip it for now. We will come back to it

In [ ]:
datapath = "/home/alpha/data/norgate/Futures_conv"
#datapath = "/Users/ageach/Documents/backup/pst_jani/norgate/Futures_conv"

check_prices_match(datapath, "SP500_micro", "20240900")

## import Norgate prices
- imports from CSV to parquet

In [ ]:
datapath = "/home/alpha/data/norgate/Futures_conv"
#datapath = "/Users/ageach/Documents/backup/pst_jani/norgate/Futures_conv"

# TODO - should adjust_hours be +23 or -1?
NORGATE_CONFIG = ConfigCsvFuturesPrices(
    input_date_index_name="Date",
    input_skiprows=0,
    input_skipfooter=0,
    input_date_format="%Y-%m-%d",
    input_column_mapping=dict(
        OPEN="Open", HIGH="High", LOW="Low", FINAL="Close", VOLUME="Volume"
    ),
    adjust_hours=-1,
)

for instr in ["SP500_micro"]:
    transfer_norgate_prices_to_db_single(instr, datapath=datapath)

In [ ]:
INSTR = "SP500_micro"

prices = diag_prices.db_futures_contract_price_data

price_dict = prices.get_merged_prices_for_instrument(INSTR)
contract_data = dict([(contractid, data) for contractid, data in price_dict.items()])
all_contract_data = dictFuturesContractPrices([(key, futuresContractPrices(x)) for key, x in contract_data.items()])
contract_prices_final = all_contract_data.final_prices()
all_finals = pd.concat(contract_prices_final, axis=1)
# all_finals = all_finals["2022-03-01":"2023-02-01"] # slice by date
# all_finals = all_finals[["20230600","20231200"]] # slice by contract
all_finals.plot(figsize=(15,9), legend=False, lw=1, grid=True, title=f"Individual contract prices for {INSTR}")

In [ ]:
roll_calendar_output = "/home/alpha/pysystemtrade/data/futures/roll_calendars_csv"
#roll_calendar_output = "/Users/ageach/Dev/work/pst_jk/data/futures/roll_calendars_csv"

build_and_write_roll_calendar(
    INSTR,
    output_datapath=roll_calendar_output,
    check_before_writing=False
)

## Check resulting roll calendar, editing if necessary
- look at the actual roll calendar CSV file
- first check the pattern of contract keys per roll date
  - you can compare against the one provided: the contract keys for a particular date in the past should be the same
  - for example, the second last row in Rob's was for 2022-03-10
    - the pattern was 20220300,20220600,20220600. The new one should be the same
- next check the last row. The date on the last row sometimes gets adjusted to the last available price row
  - if that happens, edit to be what the date sequence says it should be
    - for example: the last row for SP500_micro has date of 2024-09-06. That's because the prices were downloaded around 7 September 2024
    - edit the last row to be 2024-09-11 (just look at the previous year's September row)
- finally, if there isn't a row at the end for the next future roll date, add one
  - so for SP500_micro, there should be a future row for around 10 December 2024. But there isn't, so add it
  - copy the row for December 2023, and add it to the end. Then increment each of the years
    - so '2023-12-11,20231200,20240300,20240300' becomes '2024-12-11,20241200,20250300,20250300'


## Create multiple prices


In [ ]:
process_multiple_prices_single_instrument(INSTR, ADD_TO_CSV=True)

## Check multiple prices
- check start end dates, no gaps etc

In [ ]:
db_multiple_prices = diag_prices.db_futures_multiple_prices_data
#db_multiple_prices = multiple_prices["2023-07-01":"2023-12-31"] # slice by date
db_multiple_prices.get_multiple_prices(INSTR).plot(figsize=(15,9), legend=False, lw=1, grid=True)

## generate adjusted prices

In [ ]:
process_adjusted_prices_single_instrument(INSTR, ADD_TO_CSV=True)

## check adjusted prices

In [ ]:
db_adjusted_prices = diag_prices.db_futures_adjusted_prices_data
db_adjusted_prices.get_adjusted_prices(INSTR).plot(figsize=(15,9), legend=False, lw=1, grid=True)

## Update sampled contracts

In [ ]:
update_sampled_contracts()